In [31]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

from IPython.core.display import display, HTML
display(HTML("<style>.container { width:97% !important; }</style>"))

from sklearn.model_selection import GridSearchCV, RandomizedSearchCV, train_test_split
from sklearn.cluster import MiniBatchKMeans
from sklearn.preprocessing import RobustScaler ,StandardScaler,MinMaxScaler
from sklearn.ensemble import RandomForestRegressor

from sklearn import preprocessing
import scoring as score # para hacer los reportes de puntajes
from time import time

import jupyternotify
ip = get_ipython()
ip.register_magics(jupyternotify.JupyterNotifyMagics)

import scoring as score # para hacer los reportes de puntajes

properati = pd.read_csv('datos/properati_final.csv',error_bad_lines=False)
len(properati)

<IPython.core.display.Javascript object>

136356

# MiniKMeans

In [32]:
#paso features con strings a numericos
le_barrio = preprocessing.LabelEncoder()
barrios=properati['state_name']
le_barrio.fit(barrios)
properati['state_name'] = le_barrio.transform(barrios)

le_zona = preprocessing.LabelEncoder()
zona=properati['place_name']
le_zona.fit(zona)
properati['place_name'] = le_zona.transform(zona)

le_tipo = preprocessing.LabelEncoder()
tipos_prop=properati['property_type']
le_tipo.fit(tipos_prop)
properati['property_type'] = le_tipo.transform(tipos_prop)

In [33]:
X= zip(properati['surface_total_in_m2'],\
        properati['surface_covered_in_m2'],properati['property_type'],properati['state_name'],properati['place_name'])
scaler=preprocessing.Normalizer()
X_transformed=scaler.fit_transform(X)

In [34]:
%%notify

mini = MiniBatchKMeans()

n_clusters  = np.arange(30,300,1)

param_grid = {"n_clusters": n_clusters}

search = RandomizedSearchCV(mini, param_distributions=param_grid ,cv=5,n_iter=20) #refit deja el estimador fiteado con los mejores hiperparametros
start = time()
search.fit(X_transformed)

print("GridSearchCV duro %.2f segundos para %d candidatos a hiper-parametros."
    % (time() - start, len(search.cv_results_['params'])))
print("")
score.report_single(search.cv_results_)

GridSearchCV duro 274.21 segundos para 20 candidatos a hiper-parametros.

Puesto: 1
Promedio training score: -84.569 (std: 17.128)
Promedio validation score: -21.382 (std: 7.419)
Promedio fit time: 3.265s
Hiper-parametros: {'n_clusters': 235}

Puesto: 2
Promedio training score: -84.335 (std: 21.079)
Promedio validation score: -22.015 (std: 7.488)
Promedio fit time: 3.584s
Hiper-parametros: {'n_clusters': 268}

Puesto: 3
Promedio training score: -82.242 (std: 10.774)
Promedio validation score: -22.031 (std: 6.369)
Promedio fit time: 3.340s
Hiper-parametros: {'n_clusters': 266}

Puesto: 4
Promedio training score: -81.273 (std: 15.506)
Promedio validation score: -22.197 (std: 9.642)
Promedio fit time: 4.899s
Hiper-parametros: {'n_clusters': 277}

Puesto: 5
Promedio training score: -85.110 (std: 17.526)
Promedio validation score: -22.381 (std: 8.125)
Promedio fit time: 2.587s
Hiper-parametros: {'n_clusters': 281}

Puesto: 6
Promedio training score: -88.342 (std: 14.525)
Promedio validation

<IPython.core.display.Javascript object>

In [35]:
mejor = search.best_estimator_

In [36]:
properati["kmeans"]=search.best_estimator_.predict(X_transformed)

In [37]:
X= zip(properati['surface_total_in_m2'],\
        properati['surface_covered_in_m2'],properati['property_type'],properati['state_name'],properati['place_name'],properati["kmeans"])
y=properati['price_aprox_usd'] 

# RF

In [38]:
%%notify

rf = RandomForestRegressor(n_jobs=-1)

criterion = ['mse']
max_features =["auto","log2",None]
n_estimators = np.arange(10,200,10)

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2)

param_grid = {"criterion": criterion,"max_features":max_features,"n_estimators":n_estimators}

search = RandomizedSearchCV(rf, param_distributions=param_grid ,cv=5,n_iter=50) #refit deja el estimador fiteado con los mejores hiperparametros
start = time()
search.fit(X_train, y_train)

print("GridSearchCV duro %.2f segundos para %d candidatos a hiper-parametros."
    % (time() - start, len(search.cv_results_['params'])))
print("")
score.report_single(search.cv_results_)

KeyboardInterrupt: 

<IPython.core.display.Javascript object>

KeyboardInterrupt: 

In [46]:
%%notify

rf = RandomForestRegressor(n_jobs=-1)

criterion = ['mse']
max_features =["auto","log2"]
n_estimators = np.arange(60,70,5)

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2)

param_grid = {"criterion": criterion,"max_features":max_features,"n_estimators":n_estimators}

search = GridSearchCV(rf, param_grid=param_grid ,cv=5)
start = time()
search.fit(X_train, y_train)

print("GridSearchCV duro %.2f segundos para %d candidatos a hiper-parametros."
    % (time() - start, len(search.cv_results_['params'])))
print("")
score.report_single(search.cv_results_)

GridSearchCV duro 86.76 segundos para 4 candidatos a hiper-parametros.

Puesto: 1
Promedio training score: 0.941 (std: 0.008)
Promedio validation score: 0.675 (std: 0.115)
Promedio fit time: 2.003s
Hiper-parametros: {'max_features': 'log2', 'n_estimators': 60, 'criterion': 'mse'}

Puesto: 2
Promedio training score: 0.941 (std: 0.010)
Promedio validation score: 0.674 (std: 0.125)
Promedio fit time: 2.178s
Hiper-parametros: {'max_features': 'log2', 'n_estimators': 65, 'criterion': 'mse'}

Puesto: 3
Promedio training score: 0.929 (std: 0.016)
Promedio validation score: 0.572 (std: 0.117)
Promedio fit time: 4.197s
Hiper-parametros: {'max_features': 'auto', 'n_estimators': 65, 'criterion': 'mse'}

Puesto: 4
Promedio training score: 0.931 (std: 0.018)
Promedio validation score: 0.505 (std: 0.203)
Promedio fit time: 3.846s
Hiper-parametros: {'max_features': 'auto', 'n_estimators': 60, 'criterion': 'mse'}



<IPython.core.display.Javascript object>

In [47]:
mejor_rf = search.best_estimator_
print mejor_rf.feature_importances_

[ 0.27607906  0.47577587  0.05595004  0.04086704  0.09933401  0.05199397]


In [48]:
search.score(X_test,y_test)

0.78319269417169834

In [49]:
errores = mejor_rf.predict(X_test)-y_test
print ("Error maximo:{0}\nError minimo:{1}".format( max(abs(errores)),min(abs(errores))))
print(errores)

Error maximo:7932120.38333
Error minimo:0.0
36699      -1799.328750
69606     -66666.666667
106486    -72100.000000
62289    -133864.100918
124352      8277.500000
18253      96664.750000
118514      8974.564484
18894       3066.666667
60366      22295.000000
96917     -13361.111111
135109    -29528.414459
98261     122575.000000
108841       847.810847
50700    -304169.312169
119157      1317.613304
86351    -258466.666667
101499     48349.801587
465        44823.611111
52552     -43201.590308
112422    -67191.666667
81085      54583.333333
134628     33108.606602
116210   -124456.405556
41621      13888.760823
133338     -5644.447258
56923       9250.000000
102586     -4801.126984
96841      14526.161457
40148      29350.600212
64469       4203.691077
              ...      
64883     137561.111111
66891       -666.666667
102399     -5737.644444
57197     233848.690476
125880     -1461.045313
106789     17546.944444
103095      -616.287439
49676      -2912.966667
83793     -37616.666

In [45]:
count_max=0
max_error=1000
lista=[]
for error in errores:
    if abs(error)>=max_error:
        count_max+=1
        lista.append(abs(error))
print ("Hay {0} de {1} que tienen mas de {2} USD de error ({3}%)".format(count_max,len(errores),max_error,100 * count_max/len(errores)))

Hay 26052 de 27272 que tienen mas de 1000 USD de error (95%)
